In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df = pd.read_csv("./data/parsed_ds.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45630 entries, 0 to 45629
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   recipe_id           45630 non-null  int64 
 1   recipe_name         45630 non-null  object
 2   ingredients         45630 non-null  object
 3   ingredients_parsed  45625 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.4+ MB


In [3]:
df = df.dropna()

In [4]:
df_ing_parsed = df["ingredients_parsed"].values.astype("U")

In [5]:
df_ing_parsed

array(['sauerkraut granny smith apples onion caraway apple cider rub thai garlic powder pepper pork loin',
       'chicken wings sprigs rosemary garlic olive lemon pepper',
       'focaccia bread basil pesto chicken bell pepper onion jack cheese',
       ..., 'peaches scoops vanilla cream vanilla soy milk orange juice',
       'butter habanero peppers peaches cinnamon gallon vanilla cream crushed jalapeno habanero peanut brittle',
       'olive bulb garlic tomatoes tomatoes chiles tomato paste capers olives parsley portobello mushroom caps horseradish habanero sauce italian oregano cumin'],
      dtype='<U415')

In [6]:
#df["ingredients_parsed"].to_list()

In [7]:
vectorizer = TfidfVectorizer()

In [8]:
vectorizer.fit(df_ing_parsed)

TfidfVectorizer()

In [9]:
tfidf_recipe = vectorizer.transform(df["ingredients_parsed"])

In [11]:
print(tfidf_recipe)

  (0, 3945)	0.3329531308369232
  (0, 3593)	0.297528174400364
  (0, 3385)	0.32085017305478364
  (0, 3324)	0.3844422188658402
  (0, 3057)	0.12501529779750184
  (0, 3031)	0.20185053551072848
  (0, 2882)	0.09228101767125527
  (0, 2710)	0.1100561342109721
  (0, 2273)	0.3044709975385795
  (0, 1739)	0.297528174400364
  (0, 1635)	0.10903117396114767
  (0, 885)	0.24692286983266096
  (0, 694)	0.3294660091248842
  (0, 230)	0.24491840008216356
  (0, 222)	0.22418912847292727
  (1, 4263)	0.575856597491299
  (1, 3708)	0.48319244455714555
  (1, 3307)	0.4210994077136455
  (1, 2882)	0.16274479280297344
  (1, 2701)	0.23896172645140176
  (1, 2199)	0.2786823865503703
  (1, 1635)	0.1922850036026327
  (1, 808)	0.24364638309414482
  (2, 2906)	0.4407222209572816
  (2, 2882)	0.12806415385602504
  :	:
  (45623, 1623)	0.2991226082152092
  (45623, 1119)	0.17011736563387303
  (45623, 1071)	0.11912519560598067
  (45623, 892)	0.14889658067472133
  (45623, 601)	0.09686212550406498
  (45623, 539)	0.48840465625159796
  